The pysat library has good documentation, and you can find a comprehensive, compact tutorial directly on their GitHub repository, including Jupyter notebooks. Here are the steps to get you started:

In [1]:
!pip install python-sat

   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 9.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Visit the pysat GitHub repository for tutorials:

**GitHub Repository**: https://github.com/pysathq/pysat.

You can find example Jupyter notebooks in the examples folder:

**Examples Folder**: https://github.com/pysathq/pysat/tree/master/examples

In [2]:
from pysat.solvers import Solver

# List of common solvers you can try
solvers = ['cadical', 'glucose3', 'minisat22']

# Check which solvers are available
available_solvers = []
for solver_name in solvers:
    try:
        with Solver(name=solver_name) as solver:
            available_solvers.append(solver_name)
    except:
        print(f"{solver_name} is not available.")

print("Available solvers:", available_solvers)


cadical is not available.
Available solvers: ['glucose3', 'minisat22']


In [3]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Solver

# Create a simple CNF formula: (A OR ¬B) AND (¬A OR B)
cnf = CNF()
cnf.append([1, -2])  # (A OR ¬B)
cnf.append([-1, 2])  # (¬A OR B)

# List of available solvers you can try
solvers = ['glucose3', 'minisat22'] # omitting 'cadical'

# Solve the formula using different solvers
for solver_name in solvers:
    print(f"Solving with {solver_name}...")
    with Solver(name=solver_name, bootstrap_with=cnf) as solver:
        if solver.solve():
            print("SAT")
            print("Solution:", solver.get_model())
        else:
            print("UNSAT")
    print()


Solving with glucose3...
SAT
Solution: [-1, -2]

Solving with minisat22...
SAT
Solution: [-1, -2]



To make it easier to understand we can also print the formulas as "(q1 OR ¬q2)" 

In [4]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Solver

# Create a simple CNF formula: (A OR ¬B) AND (¬A OR B)
cnf = CNF()
cnf.append([1, -2])  # (A OR ¬B)
cnf.append([-1, 2])  # (¬A OR B)

# Print the clauses in a readable format
print("Clauses in the formula:")
for clause in cnf:
    clause_str = " OR ".join(f"q{abs(lit)}" if lit > 0 else f"¬q{abs(lit)}" for lit in clause)
    print(f"({clause_str})")
print()

# List of available solvers you can try
solvers = ['glucose3', 'minisat22']  # omitting 'cadical'

# Solve the formula using different solvers
for solver_name in solvers:
    print(f"Solving with {solver_name}...")
    with Solver(name=solver_name, bootstrap_with=cnf) as solver:
        if solver.solve():
            print("SAT")
            solution = solver.get_model()
            # Print the solution in the desired format
            for var in solution:
                var_name = f"A" if var == 1 else "B" if var == 2 else f"q{abs(var)}"
                var_value = var > 0
                print(f"{var_name} = {var_value}")
        else:
            print("UNSAT")
    print()


Clauses in the formula:
(q1 OR ¬q2)
(¬q1 OR q2)

Solving with glucose3...
SAT
q1 = False
q2 = False

Solving with minisat22...
SAT
q1 = False
q2 = False



Start with a list of n clauses, where for i=1:n, the clause is x_i_1 OR x_i_2 OR ... OR x_i_n

In [7]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def test_row_clauses_formula(n):
    """Create a formula where each row i has at least one variable x_i_j set to True, and solve it."""
    cnf = CNF()
    
    # Add clauses: each clause ensures at least one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        cnf.append(row_vars)
        print(f"Clause for row {i}: (", " OR ".join([f"x_{i}_{j}" for j in range(1, n + 1)]), ")")
    
    # Print all variables
    print("Variables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1) % n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_clauses_formula(n)


Clause for row 1: ( x_1_1 OR x_1_2 OR x_1_3 )
Clause for row 2: ( x_2_1 OR x_2_2 OR x_2_3 )
Clause for row 3: ( x_3_1 OR x_3_2 OR x_3_3 )
Variables: x_1_1, x_1_2, x_1_3, x_2_1, x_2_2, x_2_3, x_3_1, x_3_2, x_3_3

SAT: The formula is satisfiable.
x_1_1 = True
x_2_1 = True
x_3_1 = True


Create clauses for columns, so that for j=1:n, x_1_j OR x_2_j OR ...OR x_n_j

In [10]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def test_row_and_column_clauses_formula(n):
    """Create a formula where each row and each column has at least one variable set to True, and solve it."""
    cnf = CNF()
    
    # Add row clauses: each clause ensures at least one variable per row is True
    for i in range(1, n + 1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        cnf.append(row_vars)
        print(f"Row clause {i}: (", " OR ".join([f"x_{i}_{j}" for j in range(1, n + 1)]), ")")
    
    # Add column clauses: each clause ensures at least one variable per column is True
    for j in range(1, n+1):
        column_vars = [(i - 1) * n + j for i in range(1, n + 1)]
        cnf.append(column_vars)
        print(f"Column clause {j}: (", " OR ".join([f"x_{i}_{j}" for i in range(1, n + 1)]), ")")
        pass 


    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1) % n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_and_column_clauses_formula(n)


Row clause 1: ( x_1_1 OR x_1_2 OR x_1_3 )
Row clause 2: ( x_2_1 OR x_2_2 OR x_2_3 )
Row clause 3: ( x_3_1 OR x_3_2 OR x_3_3 )
Column clause 1: ( x_1_1 OR x_2_1 OR x_3_1 )
Column clause 2: ( x_1_2 OR x_2_2 OR x_3_2 )
Column clause 3: ( x_1_3 OR x_2_3 OR x_3_3 )

Variables: x_1_1, x_1_2, x_1_3, x_2_1, x_2_2, x_2_3, x_3_1, x_3_2, x_3_3

SAT: The formula is satisfiable.
x_1_2 = True
x_1_3 = True
x_2_1 = True
x_3_1 = True


Add that only one variable in a row or in a column can be evaluated to True. To do that, you need pairwise constraints for each raw and for each column.

In [27]:
# Import necessary classes
from pysat.card import CardEnc
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def test_row_and_column_clauses_formula(n):
    """Create a formula where each row and each column has exactly one variable set to True, and solve it."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    # Idea: use pysat's built in function CardEnc to build the clause which encodes (Xi1 and not Xi2 ... ant not Xin) or ... or 
    # (not X1 and not ... and not Xi(n-1) and Xin) in the CNF form
    for i in range(1, n+1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        print(row_vars)
        clauses = CardEnc.equals(
            lits = row_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)    


    # Columns, same as before
    for j in range(1, n+1):
        col_vars  = [(i - 1) * n + j for i in range(1, n + 1)]
        print(col_vars)
        clauses = CardEnc.equals(
            lits = col_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)    


    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to solve the formula
    with Minisat22(bootstrap_with=cnf) as solver:
        if solver.solve():
            print("\nSAT: The formula is satisfiable.")
            solution = solver.get_model()
            for var in solution:
                if var > 0:
                    i = (var - 1) // n + 1
                    j = (var - 1) % n + 1
                    print(f"x_{i}_{j} = True")
        else:
            print("\nUNSAT: The formula is not satisfiable.")

# Example: test a formula with n = 3
n = 3
test_row_and_column_clauses_formula(n)


[1, 2, 3]
[[1, 2, 3], [-1, -2], [-1, -3], [-2, -3]]
[4, 5, 6]
[[4, 5, 6], [-4, -5], [-4, -6], [-5, -6]]
[7, 8, 9]
[[7, 8, 9], [-7, -8], [-7, -9], [-8, -9]]
[1, 4, 7]
[[1, 4, 7], [-1, -4], [-1, -7], [-4, -7]]
[2, 5, 8]
[[2, 5, 8], [-2, -5], [-2, -8], [-5, -8]]
[3, 6, 9]
[[3, 6, 9], [-3, -6], [-3, -9], [-6, -9]]

Variables: x_1_1, x_1_2, x_1_3, x_2_1, x_2_2, x_2_3, x_3_1, x_3_2, x_3_3

SAT: The formula is satisfiable.
x_1_3 = True
x_2_2 = True
x_3_1 = True


Print all solutions found.

In [31]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def find_all_solutions(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    
    # Add row constraints: exactly one variable per row is True
    # Idea: use pysat's built in function CardEnc to build the clause which encodes (Xi1 and not Xi2 ... ant not Xin) or ... or 
    # (not X1 and not ... and not Xi(n-1) and Xin) in the CNF form
    for i in range(1, n+1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        print(row_vars)
        clauses = CardEnc.equals(
            lits = row_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)    


    # Columns, same as before
    for j in range(1, n+1):
        col_vars  = [(i - 1) * n + j for i in range(1, n + 1)]
        print(col_vars)
        clauses = CardEnc.equals(
            lits = col_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)        
    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to find all solutions
    solutions = []
    with Minisat22(bootstrap_with=cnf) as solver:
        while solver.solve():
            solution = solver.get_model()
            # Print solution
            solutions.append([var for var in solution if var > 0])
            for var in solutions[-1]:
                i = (var - 1) // n + 1
                j = (var - 1) % n + 1
                print(f"x_{i}_{j} = True", end=", ")
            print()
            
            # Add a blocking clause to exclude the current solution
            blocking_clause = [-var for var in solution if var > 0]
            solver.add_clause(blocking_clause)
    
    if not solutions:
        print("\nNo solutions found.")
    else:
        print(f"\nTotal solutions found: {len(solutions)}")

# Example: Find all solutions for n = 2
n = 2
find_all_solutions(n)


[1, 2]
[[1, 2], [-1, -2]]
[3, 4]
[[3, 4], [-3, -4]]
[1, 3]
[[1, 3], [-1, -3]]
[2, 4]
[[2, 4], [-2, -4]]

Variables: x_1_1, x_1_2, x_2_1, x_2_2
x_1_2 = True, x_2_1 = True, 
x_1_1 = True, x_2_2 = True, 

Total solutions found: 2


... add the visual output

In [33]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def print_board(solution, n):
    """Print the board configuration based on the solution."""
    board = [["." for _ in range(n)] for _ in range(n)]
    for var in solution:
        if var > 0:  # Only consider positive variables
            row = (var - 1) // n
            col = (var - 1) % n
            board[row][col] = "Q"
    
    # Print the board
    for row in board:
        print(" ".join(row))
    print()

def find_all_solutions(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    # Add row constraints: exactly one variable per row is True
    # Idea: use pysat's built in function CardEnc to build the clause which encodes (Xi1 and not Xi2 ... ant not Xin) or ... or 
    # (not X1 and not ... and not Xi(n-1) and Xin) in the CNF form
    for i in range(1, n+1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        print(row_vars)
        clauses = CardEnc.equals(
            lits = row_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)    


    # Columns, same as before
    for j in range(1, n+1):
        col_vars  = [(i - 1) * n + j for i in range(1, n + 1)]
        print(col_vars)
        clauses = CardEnc.equals(
            lits = col_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)        # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to find all solutions
    solutions = []
    with Minisat22(bootstrap_with=cnf) as solver:
        while solver.solve():
            solution = solver.get_model()
            # Print solution
            solutions.append([var for var in solution if var > 0])
            for var in solutions[-1]:
                i = (var - 1) // n + 1
                j = (var - 1) % n + 1
                print(f"x_{i}_{j} = True", end=", ")
            print("\n")
            
            # Print the board configuration
            print_board(solution, n)
            
            # Add a blocking clause to exclude the current solution
            blocking_clause = [-var for var in solution if var > 0]
            solver.add_clause(blocking_clause)
    
    if not solutions:
        print("\nNo solutions found.")
    else:
        print(f"\nTotal solutions found: {len(solutions)}")

# Example: Find all solutions for n = 2
n = 4
find_all_solutions(n)


[1, 2, 3, 4]
[[1, 2, 3, 4], [-1, -2], [-1, -3], [-1, -4], [-2, -3], [-2, -4], [-3, -4]]
[5, 6, 7, 8]
[[5, 6, 7, 8], [-5, -6], [-5, -7], [-5, -8], [-6, -7], [-6, -8], [-7, -8]]
[9, 10, 11, 12]
[[9, 10, 11, 12], [-9, -10], [-9, -11], [-9, -12], [-10, -11], [-10, -12], [-11, -12]]
[13, 14, 15, 16]
[[13, 14, 15, 16], [-13, -14], [-13, -15], [-13, -16], [-14, -15], [-14, -16], [-15, -16]]
[1, 5, 9, 13]
[[1, 5, 9, 13], [-1, -5], [-1, -9], [-1, -13], [-5, -9], [-5, -13], [-9, -13]]
[2, 6, 10, 14]
[[2, 6, 10, 14], [-2, -6], [-2, -10], [-2, -14], [-6, -10], [-6, -14], [-10, -14]]
[3, 7, 11, 15]
[[3, 7, 11, 15], [-3, -7], [-3, -11], [-3, -15], [-7, -11], [-7, -15], [-11, -15]]
[4, 8, 12, 16]
[[4, 8, 12, 16], [-4, -8], [-4, -12], [-4, -16], [-8, -12], [-8, -16], [-12, -16]]

Variables: x_1_1, x_1_2, x_1_3, x_1_4, x_2_1, x_2_2, x_2_3, x_2_4, x_3_1, x_3_2, x_3_3, x_3_4, x_4_1, x_4_2, x_4_3, x_4_4
x_1_4 = True, x_2_3 = True, x_3_2 = True, x_4_1 = True, 

. . . Q
. . Q .
. Q . .
Q . . .

x_1_2 = True

Mow add constraints that only one variable can be set to true along each diagonal from top-left position (1,1) to bottom-right position (n,1). For instance, for n=3, it should be 
Not(x_1_1 and x_2_2), Not (x_1_1 and x_3_3), Not(x_2,2 and x_3_3),
Not(x_1,2 and x_2,3), 
Not(x_2,1 and x_3,2) 

To add the constraints that ensure only one variable is True along each diagonal, we need to implement this for both types of diagonals:

Main Diagonal (Top-left to Bottom-right): These are diagonals where the difference between the row and column indices is the same

Anti-Diagonal (Top-right to Bottom-left): These diagonals have indices that sum up to the same value 


so, for diagonals, there are four areas to consider: diagonal such as x_1_1,x_2_2,x_3_3, upper and lower diagonals from this one, such as x_1_2 and x_2_3, as well as x_2_1, x_3_2. Then, there is the anti-diagonal such as x_1_3,x_2_2,x_3_1, as well as the upper and lower parts such as x_2_1 and x_1_1, as well as x_3_2 and x_2_2

In [ ]:
# Import necessary classes
from pysat.formula import CNF
from pysat.solvers import Minisat22

def add_exactly_one_constraint(cnf, variables, label):
    """Add a constraint that exactly one of the given variables can be True."""
    # At least one of them must be true
    cnf.append(variables)
    print(f"At least one in {label}: (", " OR ".join([f"x_{(var-1)//n+1}_{(var-1)%n+1}" for var in variables]), ")")
    
    # No two of them can be true at the same time (pairwise constraints)
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            cnf.append([-variables[i], -variables[j]])
            print(f"No two in {label}: ¬(x_{(variables[i]-1)//n+1}_{(variables[i]-1)%n+1} AND x_{(variables[j]-1)//n+1}_{(variables[j]-1)%n+1})")

def add_diagonal_constraints(cnf, n):
    """Add diagonal constraints to ensure only one variable can be True on each diagonal."""
    # Main diagonals (top-left to bottom-right)
    for d in range(1-n, n):
        main_diag_vars = []
        for i in range(n):
            j = i + d

            if 0 <= j < n:
                main_diag_vars.append(i * n + j + 1)

        print(f"{d}: {main_diag_vars}")

        if len(main_diag_vars) > 1:
            for i in range(len(main_diag_vars)):
                for j in range(i + 1, len(main_diag_vars)):
                    cnf.append([-main_diag_vars[i], -main_diag_vars[j]])
                    print(f"Diagonal (main): ¬(x_{(main_diag_vars[i]-1)//n+1}_{(main_diag_vars[i]-1)%n+1} AND x_{(main_diag_vars[j]-1)//n+1}_{(main_diag_vars[j]-1)%n+1})")
    
    print("====")

    # TODO: Anti-diagonals (top-right to bottom-left) (inside this function)
    for d in range(2*n - 1):
        main_antidiag_vars = []

        for i in range(n):
            j = d-i
            if 0 <= j < n:
                main_antidiag_vars.append(j* n + i + 1)

        print(f"{d}: {main_antidiag_vars}")
        if len(main_antidiag_vars) > 1:
            for i in range(len(main_antidiag_vars)):
                for j in range(i + 1, len(main_antidiag_vars)):
                    cnf.append([-main_antidiag_vars[i], -main_antidiag_vars[j]])
                    print(f"Antidiagonal (main): ¬(x_{(main_antidiag_vars[i]-1)//n+1}_{(main_antidiag_vars[i]-1)%n+1} AND x_{(main_antidiag_vars[j]-1)//n+1}_{(main_antidiag_vars[j]-1)%n+1})")

    

def print_board(solution, n):
    """Print the board configuration based on the solution."""
    board = [["." for _ in range(n)] for _ in range(n)]
    for var in solution:
        if var > 0:  # Only consider positive variables
            row = (var - 1) // n
            col = (var - 1) % n
            board[row][col] = "Q"
    
    # Print the board
    for row in board:
        print(" ".join(row))
    print()

def find_all_solutions(n):
    """Create a formula where each row and each column has exactly one variable set to True, and find all solutions."""
    cnf = CNF()
    
    for i in range(1, n+1):
        row_vars = [(i - 1) * n + j for j in range(1, n + 1)]
        print(row_vars)
        clauses = CardEnc.equals(
            lits = row_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print( clauses)    


    # Columns, same as before
    for j in range(1, n+1):
        col_vars  = [(i - 1) * n + j for i in range(1, n + 1)]
        print(col_vars)
        clauses = CardEnc.equals(
            lits = col_vars,
            bound=1,
            encoding=0
        ).clauses

        for clause in clauses:
            cnf.append(clause)
        
        print(clauses)        
    
    # TODO: Add diagonal constraints
    add_diagonal_constraints(cnf, n)
    
    # Print all variables
    print("\nVariables:", ", ".join([f"x_{i}_{j}" for i in range(1, n + 1) for j in range(1, n + 1)]))
    
    # Use Minisat22 to find all solutions
    solutions = []
    with Minisat22(bootstrap_with=cnf) as solver:
        while solver.solve():
            solution = solver.get_model()
            # Print solution
            solutions.append([var for var in solution if var > 0])
            for var in solutions[-1]:
                i = (var - 1) // n + 1
        
                j = (var - 1) % n + 1
                print(f"x_{i}_{j} = True", end=", ")
            print("\n")
            
            # Print the board configuration
            print_board(solution, n)
            
            # Add a blocking clause to exclude the current solution
            blocking_clause = [-var for var in solution if var > 0]
            solver.add_clause(blocking_clause)
    
    if not solutions:
        print("\nNo solutions found.")
    else:
        print(f"\nTotal solutions found: {len(solutions)}")

# Example: Find all solutions for n = 3
n = 4
find_all_solutions(n)


[1, 2, 3, 4]
[[1, 2, 3, 4], [-1, -2], [-1, -3], [-1, -4], [-2, -3], [-2, -4], [-3, -4]]
[5, 6, 7, 8]
[[5, 6, 7, 8], [-5, -6], [-5, -7], [-5, -8], [-6, -7], [-6, -8], [-7, -8]]
[9, 10, 11, 12]
[[9, 10, 11, 12], [-9, -10], [-9, -11], [-9, -12], [-10, -11], [-10, -12], [-11, -12]]
[13, 14, 15, 16]
[[13, 14, 15, 16], [-13, -14], [-13, -15], [-13, -16], [-14, -15], [-14, -16], [-15, -16]]
[1, 5, 9, 13]
[[1, 5, 9, 13], [-1, -5], [-1, -9], [-1, -13], [-5, -9], [-5, -13], [-9, -13]]
[2, 6, 10, 14]
[[2, 6, 10, 14], [-2, -6], [-2, -10], [-2, -14], [-6, -10], [-6, -14], [-10, -14]]
[3, 7, 11, 15]
[[3, 7, 11, 15], [-3, -7], [-3, -11], [-3, -15], [-7, -11], [-7, -15], [-11, -15]]
[4, 8, 12, 16]
[[4, 8, 12, 16], [-4, -8], [-4, -12], [-4, -16], [-8, -12], [-8, -16], [-12, -16]]
-3: [13]
-2: [9, 14]
Diagonal (main): ¬(x_3_1 AND x_4_2)
-1: [5, 10, 15]
Diagonal (main): ¬(x_2_1 AND x_3_2)
Diagonal (main): ¬(x_2_1 AND x_4_3)
Diagonal (main): ¬(x_3_2 AND x_4_3)
0: [1, 6, 11, 16]
Diagonal (main): ¬(x_1_1 AN

In [82]:
# Example: Find all solutions for n = 3
n = 10
find_all_solutions(n)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [-1, -2], [-1, -3], [-1, -4], [-1, -5], [-1, -6], [-1, -7], [-1, -8], [-1, -9], [-1, -10], [-2, -3], [-2, -4], [-2, -5], [-2, -6], [-2, -7], [-2, -8], [-2, -9], [-2, -10], [-3, -4], [-3, -5], [-3, -6], [-3, -7], [-3, -8], [-3, -9], [-3, -10], [-4, -5], [-4, -6], [-4, -7], [-4, -8], [-4, -9], [-4, -10], [-5, -6], [-5, -7], [-5, -8], [-5, -9], [-5, -10], [-6, -7], [-6, -8], [-6, -9], [-6, -10], [-7, -8], [-7, -9], [-7, -10], [-8, -9], [-8, -10], [-9, -10]]
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[[11, 12, 13, 14, 15, 16, 17, 18, 19, 20], [-11, -12], [-11, -13], [-11, -14], [-11, -15], [-11, -16], [-11, -17], [-11, -18], [-11, -19], [-11, -20], [-12, -13], [-12, -14], [-12, -15], [-12, -16], [-12, -17], [-12, -18], [-12, -19], [-12, -20], [-13, -14], [-13, -15], [-13, -16], [-13, -17], [-13, -18], [-13, -19], [-13, -20], [-14, -15], [-14, -16], [-14, -17], [-14, -18], [-14, -19], [-14, -20], [-15, -16], [-15, -17], [-15, -